## Execute the steps manually in the notebook ##

### Fill table below to submit batch job

In [ ]:

# User-defined input table
user_data = {
    "username": "test_user",
    "password": "test_password",
    "client_id": "rick sorkin",
    "User_Project_ID": "sampleproject-440900",
    "User_Dataset_ID": "user_dataset",
    "Input_Table_ID": "input_table",
    "Output_Table_ID": "output_2",
    "Model": "gpt-3.5-turbo",
    "API_key": ""
}


### These are helper functions  and libraries required to submit a job 

In [1]:
import requests
import time
import json
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [3]:

def retrieve_api_key():
    username = user_data["username"]
    password = user_data["password"]

    auth_data = {"username": username, "password": password}
    try:
        response = requests.post(f"{API_BASE_URL}/get_api_key", json=auth_data, headers=headers, timeout=10)
        if response.status_code == 200:
            api_key = response.json().get("api_key")
            logging.info(f"API key retrieved successfully: {api_key}")
            headers["x-api-key"] = api_key
            return api_key
        else:
            logging.error(f"Error retrieving API key: {response.status_code} - {response.text}")
            return None
    except requests.exceptions.RequestException as e:
        logging.error(f"Network error while retrieving API key: {e}")
        return None


In [ ]:

def get_job_data():
    return {
        "client_id": user_data["client_id"],
        "User_Project_ID": user_data["User_Project_ID"],
        "User_Dataset_ID": user_data["User_Dataset_ID"],
        "Input_Table_ID": user_data["Input_Table_ID"],
        "Output_Table_ID": user_data["Output_Table_ID"],
        "Model": user_data["Model"],
        "API_key": user_data["API_key"]
    }

In [5]:
def submit_job():
    job_data = get_job_data()

    try:
        response = requests.post(f"{API_BASE_URL}/submit_job", json=job_data, headers=headers, timeout=10)
        if response.status_code == 200:
            logging.info("Job submitted successfully.")
            return response.json()['job_id']
        else:
            logging.error(f"Error submitting job: {response.status_code} - {response.text}")
            return None
    except requests.exceptions.RequestException as e:
        logging.error(f"Network error occurred: {e}")
        return None


In [6]:
def check_job_status(job_id):
    """
    Checks the job status by calling the job_status endpoint.
    """
    try:
        response = requests.get(f"{API_BASE_URL}/job_status/{job_id}", headers=headers, timeout=10)
        if response.status_code == 200:
            return response.json()
        elif response.status_code == 404:
            logging.warning(f"Job with ID {job_id} not found.")
        else:
            logging.error(f"Failed to retrieve job status: {response.status_code} - {response.text}")
    except requests.exceptions.RequestException as e:
        logging.error(f"Network error while checking job status: {e}")
    return None


In [7]:
# Updated Exponential Backoff
def wait_for_completion(job_id, max_retries=10, backoff_time=30, max_backoff_time=3600):
    """
    Checks job status with exponential backoff.
    Exits early if the job is completed.
    """
    attempts = 0
    while attempts < max_retries:
        logging.info(f"Attempt {attempts + 1}...")
        status = check_job_status(job_id)

        if status:
            logging.info(f"Job Status Response:\n{json.dumps(status, indent=4)}")

            # Check the job's current status
            if status.get("status") == "completed":
                logging.info("✅ Job completed successfully!")
                return status
            elif status.get("status") == "processing":
                logging.info("⏳ Job is still processing. Counts so far:")
                logging.info(json.dumps(status.get("counts"), indent=4))
            elif status.get("status") == "not started":
                logging.warning("⚠️ Job has not started yet.")
            else:
                logging.warning("Unknown status: " + status.get("status", "N/A"))

        # Wait with exponential backoff
        time.sleep(backoff_time)
        backoff_time = min(backoff_time * 2, max_backoff_time)
        attempts += 1

    logging.error("❌ Maximum number of attempts reached. Job status check terminated. Please try using check_job_status_manual() for updates on job status")
    return None



In [8]:
def check_job_status_manual(api_key, job_id):
    if not api_key or not job_id:
        raise ValueError("API key or Job ID not found. Please ensure the job has been initiated.")

    status = check_job_status(job_id)
    if status:
        logging.info("🔍 Manual Check Result:")
        logging.info(json.dumps(status, indent=4))
    else:
        logging.error("🚨 Failed to retrieve job status. Check the API or Job ID.")


### Ensure that the API URL is correct with the one hosted in the cloud

In [9]:
API_BASE_URL = "http://127.0.0.1:5000" #EDIT THIS
headers = {
    "Content-Type": "application/json",
}

## Script that will submit job

In [ ]:
# Step 1: Retrieve API Key
api_key = retrieve_api_key()

# Step 2: Submit a Job
if api_key:
    job_id = submit_job()
    if job_id:
        logging.info(f"Job ID: {job_id}")

        # Step 3: Monitor Job Status
        # logging.info("🚀 Starting Exponential Backoff for Job Completion...")
        # final_status = wait_for_completion(job_id, max_retries=10, backoff_time=30, max_backoff_time=3600)
        
        # if final_status:
        #     logging.info("📊 Final Job Status Retrieved:")
        #     logging.info(json.dumps(final_status, indent=4))
        # else:
        #     logging.error("❌ Final Job Status Not Retrieved. Consider using manual check.")




2024-11-19 18:51:20,457 - INFO - API key retrieved successfully: a4f94d51-7fc7-4cd2-89c2-ab85388ec726
2024-11-19 18:51:21,095 - INFO - Job submitted successfully.
2024-11-19 18:51:21,095 - INFO - Job ID: de92ee37-17e5-4e71-b7da-24484f1a3c12
2024-11-19 18:51:21,096 - INFO - 🚀 Starting Exponential Backoff for Job Completion...
2024-11-19 18:51:21,096 - INFO - Attempt 1...
2024-11-19 18:51:21,114 - ERROR - Failed to retrieve job status: 500 - {"details":"name 'firestore' is not defined","error":"Failed to retrieve job status"}

2024-11-19 18:51:51,115 - INFO - Attempt 2...
2024-11-19 18:51:51,117 - ERROR - Failed to retrieve job status: 500 - {"details":"name 'firestore' is not defined","error":"Failed to retrieve job status"}



### Manual Job Status Check

In [ ]:
logging.info("🔎 Running Manual Check for Job Status...")
check_job_status_manual(api_key, job_id)

NameError: name 'job_id' is not defined